In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import spatial

#prepare data with generic scores

txt1 = pd.read_csv('scores/train_2022.txt', sep=" ",  header = None)
txt2 = pd.read_csv('scores/test_2022.txt', sep=" ",  header = None)
txt3 = pd.read_csv('scores/val_2022.txt', sep=" ", header = None)
data = pd.concat([txt1,txt2,txt3])
data.columns = ['image', 'score']
data.image  = data.image.str.replace('/home/ubuntu/ME-beautydatabase/images/','FaceNet_512/')
data['image'] =data['image'].str.replace(".jpg", ".csv")
data['image'] =data['image'].str.replace(".png", ".csv")
data = data.sort_values(by=['score'])
n_training, acc, mae, rmse = SVR_method(data,'score',train_number = 1000)

print(n_training, acc, mae, rmse)

/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:15: FutureWarning: The default value of regex will change from True to False in a future version.


[0.5423188405797102, 0.8791762013729978, 0.7541022876991257, 0.6411767016220887, 0.5496540375503084, 0.8500381388253244, 0.7679370191447488, 0.6493737224706841, 0.3620298113051736, 0.6429512516469039, 0.6824394070770883, 0.7404702093397746, 0.6097691894793343, 0.646400062001085, 0.6708074534161491, 0.549210887786281, 0.8390667303710783, 0.6369102021275935, 0.6713496993000099, 0.9420289855072465, 0.41064789964414233, 0.653061224489796, 0.8049613900168694, 0.778149386845039, 0.7291861761426979, 0.4670452298217364, 0.8115942028985508, 0.7815351583467527, 0.5925925925925927, 0.6498479155483987, 0.7026607789855073, 0.5668276972624798, 0.764163372859025, 0.6866311742088139, 0.639291465378422, 0.31904301817345293, 0.7428878153515834, 0.5592865105908584, 0.4089430238498562, 0.46660732259321963, 0.7991376212720086, 0.3580310436497728, 0.6128364389233955, 0.5217391304347827, 0.4991948470209339, 0.5238862050456253, 0.7478339103053061, 0.5798639141967479, 0.8030168589174802, 0.6349378881987577, 0.

In [56]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
import sklearn.metrics as metrics
from sklearn import preprocessing

#SVR on 512 facenet

def SVR_method(df,column, train_number = 100):
    
       #df['True'] = df[column].apply(lambda x: True if (x > df[column].mean()) else False)
       df = df.sample(frac=1).reset_index(drop=True)
       
       min_max_scaler = preprocessing.MinMaxScaler()
    
       df[column] = min_max_scaler.fit_transform(df[column].values.reshape(-1,1))  
    
       df2 = pd.DataFrame(columns = ['image', column, 'features'])
       
       for index, row in df.iterrows():
            
          try:
        
             v1 = np.loadtxt(row['image'], delimiter=',')
             df2.loc[len(df2)]=[row['image'],row[column], v1]
             #print(v1.shape)
          except:
             pass
       
       #df2 = df2.drop(['image'], axis=1)
       
       
       train= df2[:train_number]
       test= df2[train_number:]
       #one_test = test.iloc[[55]]
       regressor = SVR(kernel = 'rbf') 
       train_X = min_max_scaler.fit_transform(train['features'].values.tolist())    
       test_X = min_max_scaler.fit_transform(test['features'].values.tolist())
       train_Y = (train[column].values.tolist()) 
       test_Y = (test[column].values.tolist())
        
       print(train_Y[:60])
        
       regressor.fit(train_X, train_Y)
       #w = regressor.coef_.dot(np.asarray(one_test['features'].values.tolist()).reshape(512))
       result = regressor.predict(test_X) 
       #print(result[:100])
       #print(result[55],w+regressor.intercept_)
       mae =  metrics.mean_absolute_error(test_Y, result)
       rmse = metrics.mean_squared_error(test_Y, result)
       return train_number, pearsonr(test_Y, result), mae, rmse
                                                   
             

In [19]:
#prepare the personal score data

data = pd.read_excel('scores/date_scores_all_2022.xlsx')
data = data.rename(columns={"maleimage": "image"})
data = data.drop(['Unnamed: 0','Unnamed: 0.1', 'average'], axis=1)
data['image'] =data['image'].str.replace('/home/ubuntu/ME-beautydatabase/images','FaceNet_512')
data['image'] =data['image'].str.replace(".jpg", ".csv")
data['image'] =data['image'].str.replace(".png", ".csv")
data = data[data['image'].notna()]


data_clean = pd.DataFrame(columns=data.columns)
        
for index, row in data.iterrows():
            
           try:
             #print(row)
             v1 = np.loadtxt(row['image'], delimiter=',')
             data_clean.loc[len(data_clean)] = row  
           except:
             #print('no')
             pass

df_acc = pd.DataFrame(columns = ['user', 'img','PC', 'mae', 'rmse'])

for column in data_clean:
    
    if column != "image":
       
       #print(column)
       df = data_clean[['image',column]]
       df = df.dropna() 
        
       
        
       train_number=10   
    
       if len(df) > train_number + 30:
          n_training, acc, mae, rmse = SVR_method(df, column, train_number)
          df_acc.loc[len(df_acc)] = [column, n_training, acc[0], mae, rmse]
       else:
          pass   
        
print(df_acc.mean())
print(df_acc)
       #print(len(train),(r[0,0]+r[1,1])/r.sum()) 
    

/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/ipykernel/__main__.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


img     10.000000
PC       0.173183
mae      2.055974
rmse     7.311193
dtype: float64
                              user img        PC       mae      rmse
0    labelcaucasian_female_29.xlsx  10  0.130389  0.639519  1.309394
1    labelcaucasian_female_27.xlsx  10  0.403891  1.504695  3.279088
2    labelcaucasian_female_31.xlsx  10  0.325131  1.281521  2.972515
3        labelasian_female_22.xlsx  10  0.183481  1.241882  3.993732
4    labelcaucasian_female_32.xlsx  10  0.087930  2.027316  6.492789
..                             ...  ..       ...       ...       ...
269                  AY0758IITCGOZ  10  0.046613  1.225585  2.838388
270                  AY7RI824FORAE  10  0.192791  2.355160  8.480514
271                  AY9TXXA0DTKUS  10  0.133564  1.756366  7.112201
272                  AZ8KL6F16ZS7G  10  0.412813  1.771275  4.682676
273                  AZD9Z5B2U40G2  10  0.036570  1.730421  4.895782

[274 rows x 5 columns]


In [15]:
df_acc[50:90]

,user,img,PC,mae,rmse
50,A24EYX9Y59UEIZ,100,0.209350,2.962827,14.016370
51,A251K96S0ZIQPX,100,0.313936,2.319881,8.262769
52,A251PGFFD9306F,100,0.393804,2.426174,8.546316
53,A25SH3L1UR6Y1Z,100,0.600501,2.503198,11.839562
54,A27QN0STWBD2CU,100,0.220742,1.400438,4.310554
55,A287X3UTSM9XPY,100,-0.119368,1.469062,3.466411
56,A28BWXVCFZI7EU,100,0.388817,1.637113,4.187609
57,A29XYWTD63Z7BS,100,0.463059,2.022989,5.540608
58,A2A1FS0KVLJ9ZT,100,0.100881,3.514360,15.810759
59,A2AJIPVZWKKZJO,100,0.465957,0.984790,1.624747


In [ ]:
def average_face(train, attr = True):
   
   #create a vector with average attr or not face

   df0 = pd.DataFrame()
   df1 = pd.DataFrame()
   v = [0] * 512
   ind = 0

   train2 = train.loc[train['True'] == attr]

   for index, row in train2.iterrows():
    
      row['image'] = row['image'].replace(".jpg", ".csv")
      row['image'] = row['image'].replace(".png", ".csv")

      try:

         v1 = np.loadtxt(row['image'], delimiter=',')
         v=np.add (v,v1)
         ind += 1
        
      except:
        
         pass
    

   avrg = v/ind

   return avrg, ind

def average_face_method(df,column, train_number = 100):
    
       df['True'] = df[column].apply(lambda x: True if (x > df[column].mean()) else False)
       df = df.sample(frac=1).reset_index(drop=True)
       train= df[:train_number]
       test= df[train_number:]

       attr,x1 = average_face(train, True)
       not_attr,x2 = average_face(train, False)
        
       result = pd.DataFrame(columns=['pred', 'real'])
       test = test.sample(frac=1).reset_index(drop=True)
     
       for index, row in test.iterrows():
          #print(row['image'])
          try:
        
             v1 = np.loadtxt(row['image'], delimiter=',')
             d1 = spatial.distance.euclidean(v1, attr)
             d2 = spatial.distance.euclidean(v1, not_attr)
     
             #print(d1,d2)
             if d1>=d2: 
            
                result.loc[len(result)]=[False,row['True']]    
                
             else: 

                result.loc[len(result)]=[True,row['True']]
            
    
          except:
             pass
       #print(result)
       r = result.groupby(["pred", "real"]).size()

       n_train_images = x1+x2
       
       try:
          acc = (r[0,0]+r[1,1])/r.sum() 
       except:
          acc = 0
       
       return n_train_images, acc